#### 使用GWAC数据

In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import add,Input,Conv1D,Activation,Flatten,Dense
from sklearn.metrics import classification_report
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
def loaddata(): 
    train_fn_0 = '../../dataset/train_0.csv'
    train_fn_1 = '../../dataset/train_1.csv'
    test_fn_0 = '../../dataset/test_0.csv'
    test_fn_1 = '../../dataset/test_1.csv'
    # 加载训练集
    train_0 = np.loadtxt(train_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    train_1 = np.loadtxt(train_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    train = np.concatenate((train_0,train_1),axis=0)
    np.random.shuffle(train)
    # 加载测试集
    test_0 = np.loadtxt(test_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    test_1 = np.loadtxt(test_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    test = np.concatenate((test_0, test_1), axis=0)
    np.random.shuffle(test)
    # 整理数据维度
    x_train = train[:,0:30][:,:,np.newaxis]
    x_test = test[:,0:30][:,:,np.newaxis]
    y_train = train[:,30].astype(np.int)
    y_train = np.eye(2)[y_train]
    y_test = test[:,30].astype(np.int)
    y_test = np.eye(2)[y_test]
    print(x_train.shape)
    print(y_train.shape)
    return x_train,y_train,x_test,y_test

In [4]:
#残差块
def ResBlock(x,filters,kernel_size,dilation_rate):
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate,activation='relu')(x) #第一卷积
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate)(r) #第二卷积
    if x.shape[-1]==filters:
        shortcut=x
    else:
        shortcut=Conv1D(filters,kernel_size,padding='same')(x)  #shortcut（捷径）
    o=add([r,shortcut])
    o=Activation('relu')(o)  #激活函数
    return o
 
#序列模型
inputs=Input(shape=(30,1))
x=ResBlock(inputs,filters=32,kernel_size=3,dilation_rate=1)
x=ResBlock(x,filters=32,kernel_size=3,dilation_rate=2)
x=ResBlock(x,filters=16,kernel_size=3,dilation_rate=4)
x=Flatten()(x)
x=Dense(2,activation='softmax')(x)
model=Model(inputs=inputs,outputs=x)
#查看网络结构
model.summary()
#编译模型
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


#训练模型
train_x,train_y,test_x,test_y=loaddata()
start_time = timer()
model.fit(train_x,train_y,validation_split=0.3,batch_size=500,epochs=10)
current_time = timer()
print("train_time",(current_time-start_time)/10)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 30, 32)       128         input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 30, 32)       3104        conv1d_8[0][0]                   
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 30, 32)       128         input_2[0][0]                    
____________________________________________________________________________________________

In [5]:
#评估模型
pre=model.evaluate(test_x,test_y,batch_size=500,verbose=2)
print('test_loss:',pre[0],'- test_acc:',pre[1])

TCN_y_pred=model.predict(test_x, batch_size=500)
print(classification_report(test_y, TCN_y_pred.round(),digits=4))

20/20 - 0s - loss: 0.0166 - accuracy: 0.9952
test_loss: 0.01663932390511036 - test_acc: 0.9951604604721069
              precision    recall  f1-score   support

           0     1.0000    0.9933    0.9966      6872
           1     0.9828    1.0000    0.9913      2633

   micro avg     0.9952    0.9952    0.9952      9505
   macro avg     0.9914    0.9967    0.9940      9505
weighted avg     0.9952    0.9952    0.9952      9505
 samples avg     0.9952    0.9952    0.9952      9505



In [6]:
# 测试10次的运行时间
test_time_tcn = []
for i in range(10):
    start_time = timer()
    print(start_time)
    model.predict(test_x, batch_size=500)
    current_time = timer()
    print(current_time)
    test_time_tcn.append(current_time-start_time)
    print(current_time-start_time)
    i = i+1
print(test_time_tcn)
print(np.mean(test_time_tcn))

48868430.846562006
48868431.16365225
0.31709024310112
48868431.16374283
48868431.44080075
0.2770579159259796
48868431.44091488
48868431.71859448
0.27767959982156754
48868431.718690686
48868432.0013223
0.28263161331415176
48868432.00140891
48868432.26956034
0.2681514248251915
48868432.26964952
48868432.55726721
0.28761769086122513
48868432.55735099
48868432.86706437
0.30971337854862213
48868432.86715589
48868433.152763434
0.28560754656791687
48868433.15285186
48868433.435434096
0.28258223831653595
48868433.43552084
48868433.6939803
0.2584594562649727
[0.31709024310112, 0.2770579159259796, 0.27767959982156754, 0.28263161331415176, 0.2681514248251915, 0.28761769086122513, 0.30971337854862213, 0.28560754656791687, 0.28258223831653595, 0.2584594562649727]
0.28465911075472833


In [4]:

TCN(train_x,train_y,test_x,test_y)

(38022, 30, 1)
(38022, 2)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 30, 32)       128         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 32)       3104        conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30, 32)       128         input_1[0][0]                    
____________________________________________________________________

#### 使用mnist作为数据集进行学习和测试

https://pypi.org/project/keras-tcn/

https://github.com/philipperemy/keras-tcn

https://blog.csdn.net/m0_37602827/article/details/104883494

https://zhuanlan.zhihu.com/p/349613716

输入的维度分别表示（样本数，时间步，每一时间步的特征）

In [1]:
# 使用tensorflow1将数据集下载下来，然后在使用tensorflowCPU （2.2.0）运行后面的代码
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets ( "MNIST_data/", one_hot=True)

/home/wamdm/anaconda3/envs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wamdm/anaconda3/envs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wamdm/anaconda3/envs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wamdm/anaconda3/envs/tensorf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
train_x,train_y=mnist.train.images.reshape(-1,28,28),mnist.train.labels,
valid_x,valid_y=mnist.validation.images.reshape(-1,28,28),mnist.validation.labels,
test_x,test_y=mnist.test.images.reshape(-1,28,28),mnist.test.labels
print(train_x.shape,train_y.shape,valid_x.shape,valid_y.shape,test_x.shape,test_y.shape)

(55000, 28, 28) (55000, 10) (5000, 28, 28) (5000, 10) (10000, 28, 28) (10000, 10)


In [5]:
# from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from keras.models import Model
from keras.layers import add,Input,Conv1D,Activation,Flatten,Dense
 
#载入数据
def read_data(path):
#     mint=tf.keras.datasets.mnist
#     (train_images,train_labels),(test_images,test_labels)=keras.datasets.mnist.load_data()
    mnist=input_data.read_data_sets(path,one_hot=True)
    train_x,train_y=mnist.train.images.reshape(-1,28,28),mnist.train.labels,
    valid_x,valid_y=mnist.validation.images.reshape(-1,28,28),mnist.validation.labels,
    test_x,test_y=mnist.test.images.reshape(-1,28,28),mnist.test.labels
    return train_x,train_y,valid_x,valid_y,test_x,test_y
 

#残差块
def ResBlock(x,filters,kernel_size,dilation_rate):
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate,activation='relu')(x) #第一卷积
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate)(r) #第二卷积
    if x.shape[-1]==filters:
        shortcut=x
    else:
        shortcut=Conv1D(filters,kernel_size,padding='same')(x)  #shortcut（捷径）
    o=add([r,shortcut])
    o=Activation('relu')(o)  #激活函数
    return o
 
#序列模型
def TCN(train_x,train_y,valid_x,valid_y,test_x,test_y):
    inputs=Input(shape=(28,28))
    x=ResBlock(inputs,filters=32,kernel_size=3,dilation_rate=1)
    x=ResBlock(x,filters=32,kernel_size=3,dilation_rate=2)
    x=ResBlock(x,filters=16,kernel_size=3,dilation_rate=4)
    x=Flatten()(x)
    x=Dense(10,activation='softmax')(x)
    model=Model(input=inputs,output=x)
    #查看网络结构
    model.summary()
    #编译模型
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    #训练模型
    model.fit(train_x,train_y,batch_size=500,nb_epoch=30,verbose=2,validation_data=(valid_x,valid_y))
    #评估模型
    pre=model.evaluate(test_x,test_y,batch_size=500,verbose=2)
    print('test_loss:',pre[0],'- test_acc:',pre[1])
    
     
train_x,train_y,valid_x,valid_y,test_x,test_y=read_data('MNIST_data')
TCN(train_x,train_y,valid_x,valid_y,test_x,test_y)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


/home/wamdm/anaconda3/envs/tensorflow1/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/home/wamdm/anaconda3/envs/tensorflow1/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 28, 28)        0                                            
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 28, 32)        2720        input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 28, 32)        3104        conv1d_1[0][0]                   
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 28, 32)        2720        input_1[0][0]                    
___________________________________________________________________________________________